<H1 style="text-align: center;">Hypoglycemia Detection</H1>
###Imports

In [180]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import math
from datetime import datetime

###Loading CSVs Into Pandas DataFrame

In [234]:
#Read files using relative local path
fileNames = glob.glob("../Data/d*")

df = pd.DataFrame([]) # defining a dataframe
for i in range(len(fileNames)):
    # Read files and save as dataframe.
    frames = pd.read_csv(fileNames[i], sep='\t', header=None, names=['Date', 'Time',  'Code','Blood_Glucose'])
    
    #Create patient id column from the name of the file. This uniquely identifies a patient.
    frames['Patient_ID']= fileNames[i][-2:]
    #Fill empty dates by ffill as there are many glucose readings in a day.
    frames['Date'] = frames['Date'].fillna(method='ffill')
  
    #Creating a master dataframe.
    df=df.append(frames)

#Change type of Code column to category.
df['Code'] = df['Code'].astype('category')

#Convert blood glucose column to numeric. By default it's object.
df['Blood_Glucose'] = pd.to_numeric(df['Blood_Glucose'], errors='coerce')

#Convert Code column to String.
df['Code'] = df['Code'].astype(str) 

#Print all distinct code values for validation. We can see few codes that are not found in the website data description
df['Code'].unique()

array(['58', '33', '34', '60', '62', '48', '65', '67', '56', '57', '64',
       '68', '71', '72', '70', '63', '61', '69', '66', '35', '59', '36',
       '0', '4'], dtype=object)

###Dataframe Before Cleanup
**Date :** Date of blood glucose measurement
**Time :** Time of blood glucose measurement
**Code :** Event during blood glucose measurement __eg: before breakfast, after exercise, before snacks etc.__
**Blood_Glucose :** Blood glucose levels obtained during the measurement.
**Patient ID :** Unique ID denoting a patient. This field was generated by the code while loading the file.

In [235]:
df.head()

,Date,Time,Code,Blood_Glucose,Patient_ID
0,05-20-1991,08:00,58,101.0,31
1,05-20-1991,08:00,33,5.0,31
2,05-20-1991,08:00,34,27.0,31
3,05-20-1991,12:00,60,89.0,31
4,05-20-1991,12:00,33,3.0,31


In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29330 entries, 0 to 299
Data columns (total 5 columns):
Date             29330 non-null object
Time             29330 non-null object
Code             29330 non-null object
Blood_Glucose    29278 non-null float64
Patient_ID       29330 non-null object
dtypes: float64(1), object(4)
memory usage: 1.3+ MB


###Data Wrangling
**Step 1 :** Remove rows where blood glucose is null. As no validations can be performed with out it.

In [237]:
df.drop(df.loc[df['Blood_Glucose'].isnull()].index, inplace=True)

df.head()

,Date,Time,Code,Blood_Glucose,Patient_ID
0,05-20-1991,08:00,58,101.0,31
1,05-20-1991,08:00,33,5.0,31
2,05-20-1991,08:00,34,27.0,31
3,05-20-1991,12:00,60,89.0,31
4,05-20-1991,12:00,33,3.0,31


**Step 2 :** Add new column code description. The code is a number that defines the event during which the blood glucose
was measured. It is more readable to have description than numbers.

Create a dictionary with code and its description and map it.

The codes are got from the data description itself. The data description does not give the details on all codes.They all are classified as unknowns.

In [238]:
code_dict ={"0" : "UNKNOWN",
            "4" : "UNKNOWN", 
            "33" : "REGULAR_INSULIN_DOSE", 
            "34" : "NPH_INSULIN_DOSE", 
            "35" : "ULTRALENTE_INSULIN_DOSE", 
            "36" : "UNKNOWN",
            "48" : "UNKNOWN", 
            "56" : "UNKNOWN",
            "57" : "UNKNOWN", 
            "58" : "PRE_BREAKFAST",
            "59" : "POST_BREAKFAST", 
            "60" : "PRE_LUNCH",
            "61" : "POST_LUNCH",
            "62" : "PRE_SUPPER",
            "63" : "POST_SUPPER",
            "64" : "PRE_SNACK", 
            "65" : "HYPOGLYCEMIC_SYMPTOMS", 
            "66" : "TYPICAL_MEAL_INJESTION", 
            "67" : "MORE_MEAL_INJESTION", 
            "68" : "LESS_MEAL_INJESTION", 
            "69" : "TYPICAL_EXERCISE",
            "70" : "MORE_EXERCISE",
            "71" : "LESS_EXERCISE",
            "72" : "UNKNOWN"}

df['Code_Description'] = df['Code'].map(code_dict)

df.head()

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description
0,05-20-1991,08:00,58,101.0,31,PRE_BREAKFAST
1,05-20-1991,08:00,33,5.0,31,REGULAR_INSULIN_DOSE
2,05-20-1991,08:00,34,27.0,31,NPH_INSULIN_DOSE
3,05-20-1991,12:00,60,89.0,31,PRE_LUNCH
4,05-20-1991,12:00,33,3.0,31,REGULAR_INSULIN_DOSE


**Step 3:** Clean date and time
**Procedure :** Create new Date_Time column from Date and time. the 'to_datetime' method convert's the string to time if the string is a valid date else fills the cell as 'NaT'. We will use 'NaT' as a flag and fix the date and time step by step.


In [239]:
df['Date_Time'] = pd.to_datetime(df['Date'].str.strip()+" "+df['Time'].str.strip(), 
                                 format="%m-%d-%Y %H:%M", errors="coerce")

df.head()

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
0,05-20-1991,08:00,58,101.0,31,PRE_BREAKFAST,1991-05-20 08:00:00
1,05-20-1991,08:00,33,5.0,31,REGULAR_INSULIN_DOSE,1991-05-20 08:00:00
2,05-20-1991,08:00,34,27.0,31,NPH_INSULIN_DOSE,1991-05-20 08:00:00
3,05-20-1991,12:00,60,89.0,31,PRE_LUNCH,1991-05-20 12:00:00
4,05-20-1991,12:00,33,3.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00


**A sample of rows with invalid data.**

In [240]:
df.loc[df.isnull().any(axis=1)].head()

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
792,04-09-1991,56:35,58,237.0,67,PRE_BREAKFAST,NaT
793,04-09-1991,56:35,33,16.0,67,REGULAR_INSULIN_DOSE,NaT
794,04-09-1991,56:35,34,40.0,67,NPH_INSULIN_DOSE,NaT
893,04-21-1991,188:00,62,128.0,67,PRE_SUPPER,NaT
894,04-21-1991,188:00,33,14.0,67,REGULAR_INSULIN_DOSE,NaT


**Step 4 :** As codes(eg: before breakfast) happen in similar time for all the days, our logic is to get a valid time from another row for the same patient. To achieve this we create another dataframe df_agg by grouping df by patient id
and code. This data frame will be used to create a dictionary to simplify the next steps.

In [242]:
#This function return's the median time from a pandas series
def medianTime(groupSeries):
    groupSeries = np.sort(pd.to_datetime(groupSeries, format="%H:%M").dt.time)
    return str(groupSeries[int(len(groupSeries)/2)])


#Here we have used np.min to aggregate instead of np.mean as the difference between the two is negligible.
df_agg = pd.DataFrame(df.loc[df['Date_Time'].notnull()].groupby(['Patient_ID', 'Code'])['Time'].agg(medianTime))
df_agg = df_agg.reset_index()

dic = {row['Patient_ID']+"_"+row['Code']:  row['Time']  for i,row in df_agg.iterrows()}

dic['01_33']#"01" is patient id and "33" is the code.

'12:32:00'

**Step 5:** Use the above dictionary to replace the invalid times. 
**Repeat to_datetime method to see if all times are valid.**

In [245]:
df.loc[df.isnull().any(axis=1),'Time'] = df.loc[df.isnull().any(axis=1)].apply(lambda row : dic[row['Patient_ID']+"_"+row['Code']], axis=1)


df['Date_Time'] = pd.to_datetime(df['Date'].str.strip()+" "+df['Time'].str.strip(), 
                                 format="%m-%d-%Y %H:%M:%S", errors="coerce")


df.loc[df.isnull().any(axis=1)]

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
363,06-31-1991,06:50:00,58,149.0,20,PRE_BREAKFAST,NaT
364,06-31-1991,12:05:00,33,4.0,20,REGULAR_INSULIN_DOSE,NaT
365,06-31-1991,06:50:00,34,24.0,20,NPH_INSULIN_DOSE,NaT
366,06-31-1991,12:00:00,60,162.0,20,PRE_LUNCH,NaT
367,06-31-1991,12:05:00,33,5.0,20,REGULAR_INSULIN_DOSE,NaT
368,06-31-1991,18:15:00,62,213.0,20,PRE_SUPPER,NaT
369,06-31-1991,12:05:00,33,11.0,20,REGULAR_INSULIN_DOSE,NaT


**Step 6 :**We can see that there are still invalid date_time. From the data above it's clear that this time its the date that is causing the problem. So let's sample the patient's data by closest dates to see if there are any anomalies.

In [246]:
df.loc[((df['Patient_ID']=='20') & (df['Date']=='06-29-1991'))]

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
355,06-29-1991,06:50:00,58,379.0,20,PRE_BREAKFAST,1991-06-29 06:50:00
356,06-29-1991,12:05:00,33,9.0,20,REGULAR_INSULIN_DOSE,1991-06-29 12:05:00
357,06-29-1991,06:50:00,34,24.0,20,NPH_INSULIN_DOSE,1991-06-29 06:50:00
358,06-29-1991,12:00:00,60,180.0,20,PRE_LUNCH,1991-06-29 12:00:00
359,06-29-1991,12:05:00,33,5.0,20,REGULAR_INSULIN_DOSE,1991-06-29 12:05:00
360,06-29-1991,18:15:00,62,119.0,20,PRE_SUPPER,1991-06-29 18:15:00
361,06-29-1991,12:05:00,33,9.0,20,REGULAR_INSULIN_DOSE,1991-06-29 12:05:00
362,06-29-1991,22:30:00,48,177.0,20,UNKNOWN,1991-06-29 22:30:00


In [247]:
df.loc[((df['Patient_ID']=='20') & (df['Date']=='06-30-1991'))]

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time


In [248]:
df.loc[((df['Patient_ID']=='20') & (df['Date']=='07-01-1991'))]

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
370,07-01-1991,06:50:00,58,174.0,20,PRE_BREAKFAST,1991-07-01 06:50:00
371,07-01-1991,12:05:00,33,5.0,20,REGULAR_INSULIN_DOSE,1991-07-01 12:05:00
372,07-01-1991,06:50:00,34,24.0,20,NPH_INSULIN_DOSE,1991-07-01 06:50:00
373,07-01-1991,12:00:00,60,62.0,20,PRE_LUNCH,1991-07-01 12:00:00
374,07-01-1991,12:05:00,33,3.0,20,REGULAR_INSULIN_DOSE,1991-07-01 12:05:00
375,07-01-1991,18:15:00,62,294.0,20,PRE_SUPPER,1991-07-01 18:15:00
376,07-01-1991,12:05:00,33,12.0,20,REGULAR_INSULIN_DOSE,1991-07-01 12:05:00
377,07-01-1991,22:30:00,48,111.0,20,UNKNOWN,1991-07-01 22:30:00


**Step 7:** By comparing the above 3 values, we can see that there are no records for 06/30. We can assume that its a typo and replace 06/31 with 06/30. We will again repeat the date time conversion

In [249]:
df.loc[df.isnull().any(axis=1), 'Date'] = '06-30-1990'

df['Date_Time'] = pd.to_datetime(df['Date'].str.strip()+" "+df['Time'].str.strip(), 
                                 format="%m-%d-%Y %H:%M:%S", errors="coerce")

###Data Wrangling Complete

**Finally we see that all data are valid.**

In [250]:


df.loc[df.isnull().any(axis=1)]

,Date,Time,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time


###Memory cleanup

In [251]:
df = df.drop(['Date', 'Time'], axis=1)

del df_agg

del dic

###Cleaned Data

In [252]:


df.head()

,Code,Blood_Glucose,Patient_ID,Code_Description,Date_Time
0,58,101.0,31,PRE_BREAKFAST,1991-05-20 08:00:00
1,33,5.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00
2,34,27.0,31,NPH_INSULIN_DOSE,1991-05-20 08:00:00
3,60,89.0,31,PRE_LUNCH,1991-05-20 12:00:00
4,33,3.0,31,REGULAR_INSULIN_DOSE,1991-05-20 12:00:00


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28253 entries, 0 to 299
Data columns (total 5 columns):
Code                28253 non-null object
Blood_Glucose       28253 non-null float64
Patient_ID          28253 non-null object
Code_Description    28253 non-null object
Date_Time           28253 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.3+ MB


###Write dataframe to CSV

In [257]:
df.to_csv('../Data/Cleaned/data.csv', index=False)